In [ ]:
! pip install -U spacy -q

In [ ]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.7.2                         
Location         /usr/local/lib/python3.10/dist-packages/spacy
Platform         Linux-6.1.58+-x86_64-with-glibc2.35
Python version   3.10.12                       
Pipelines        en_core_web_sm (3.7.1)        



In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

In [ ]:
import json
f = open('/content/master.json')
TRAIN_DATA = json.load(f)

In [ ]:
for text, annot in tqdm(TRAIN_DATA['annotations']):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

100%|██████████| 200/200 [00:01<00:00, 112.70it/s]


In [ ]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency -F

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     68.67    0.10    0.06    0.24    0.00
  1     200         63.62   3598.39   63.90   77.52   54.35    0.64
  2     400        113.22    768.52   87.28   88.22   86.35    0.87
  3     600        171.62    518.42   89.75   87.40   92.24    0.90
  4     800        250.04    309.10   90.40   83.89   98.00    0.90
  5    1000        180.79    266.48   96.24   96.13   96.35    0.96
  6    1200        317.10    183.01   95.62   95.01   96.24    0.96
  7    1400        238.87    170.29   97.48   97.08   97.88    0.97
  8    1600        215.97    140.43 

In [ ]:
nlp_ner = spacy.load("/content/model-last")

In [ ]:
doc = nlp_ner('''Bhargavi Lives in Ville Parle and Alan lives in Andheri''') # input sample text

In [ ]:
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter

In [ ]:
loc_entities = [ent.text for ent in doc.ents]

print(loc_entities)

['Ville Parle', 'Andheri']


In [ ]:
import pandas as pd
df = pd.DataFrame(loc_entities)

In [ ]:
df

,0
0,Ville Parle
1,Andheri


In [ ]:
!zip -r /content/file.zip /content/model-last

  adding: content/model-last/ (stored 0%)
  adding: content/model-last/tok2vec/ (stored 0%)
  adding: content/model-last/tok2vec/model (deflated 8%)
  adding: content/model-last/tok2vec/cfg (stored 0%)
  adding: content/model-last/vocab/ (stored 0%)
  adding: content/model-last/vocab/key2row (stored 0%)
  adding: content/model-last/vocab/strings.json (deflated 72%)
  adding: content/model-last/vocab/vectors.cfg (stored 0%)
  adding: content/model-last/vocab/vectors (deflated 45%)
  adding: content/model-last/vocab/lookups.bin (stored 0%)
  adding: content/model-last/meta.json (deflated 56%)
  adding: content/model-last/config.cfg (deflated 61%)
  adding: content/model-last/ner/ (stored 0%)
  adding: content/model-last/ner/moves (deflated 40%)
  adding: content/model-last/ner/model (deflated 7%)
  adding: content/model-last/ner/cfg (deflated 33%)
  adding: content/model-last/tokenizer (deflated 81%)


In [ ]:
from google.colab import files
files.download("/content/file.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>